# Fig. S4 - Participant electrode coverage

In [ ]:
import json
import os
import pickle

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from sylseq_paper.plotting import default_plot_settings, ucsf_sequential_color_palette as colors

default_plot_settings()

In [ ]:
# Set up paths
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
img_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'imaging'))
subj_img_path = os.path.join(img_dir, '{}_{}_brain_2D.png')

# Load bad channels
with open(os.path.join(data_dir, 'bad_channels.pkl'), 'rb') as f:
    bad_channels = pickle.load(f)
    
lateral_and_medial_subjects = ['EC217', 'EC240', 'EC241', 'EC276', 'EC289']
lateral_subjects = ['EC219', 'EC223', 'EC237', 'EC253', 'EC254', 'EC260', 'EC263', 'EC267', 'EC282']

exclude_areas = ['superiorparietal', 'inferiortemporal', 'paracentral', 'medial']

sig_thresh = 0.05

all_edf = pd.read_hdf(os.path.join(data_dir, 'all_anatomical_info.h5'))
all_edf = all_edf.loc[~all_edf.location.isin(exclude_areas)]

In [ ]:
# Load dataframe electrode information
with open(os.path.join(data_dir, 'fig1_ndf_sustained_se.pkl'), 'rb') as f:
    results = pickle.load(f)
    neural_df = results['neural_df']
    main_sustained_se = results['sustained_se']

supp_sustained_se = []
supp_sig_anytime = []

# these subjects were excluded from fig 1 analyses so cluster assignments in separate file
for subject in ['EC267', 'EC282', 'EC289']:
    with open(os.path.join(data_dir, f'{subject}_supp_analyses.pkl'), 'rb') as f:
        results = pickle.load(f)
        supp_sustained_se.extend([f'{subject}_{e}' for e in results['sustained_elecs']])
        supp_sig_anytime.extend([f'{subject}_{e}' for e in results['any_period_sig_elecs']])

all_sustained_elecs = np.unique(np.concatenate([main_sustained_se, supp_sustained_se]))
all_sustained_elecs = all_edf.loc[all_edf.subject_electrode.isin(all_sustained_elecs)].subject_electrode.values

main_sig_anytime = neural_df.loc[neural_df.significant_above_baseline_anytime].subject_electrode.unique()
all_sig_anytime = np.unique(np.concatenate([main_sig_anytime, supp_sig_anytime]))

In [ ]:
def plot_elecs(ax=None, df=None):
    "plot any sig elecs, sustained elecs, rest of the elecs"
    
    any_df = df.loc[(df.subject_electrode.isin(all_sig_anytime)) & (~df.subject_electrode.isin(all_sustained_elecs))]
    sus_df = df.loc[(df.subject_electrode.isin(all_sig_anytime)) & (df.subject_electrode.isin(all_sustained_elecs))]
    rest_df = df.loc[(~df.subject_electrode.isin(all_sig_anytime)) & (~df.subject_electrode.isin(all_sustained_elecs))]
    
    ax.scatter(rest_df.x, rest_df.y, s=1, color='k', alpha=0.35)
    ax.scatter(sus_df.x, sus_df.y, s=10, edgecolor='none', facecolor='orange', alpha=1.0)
    ax.scatter(sus_df.x, sus_df.y, s=10, edgecolor='k', facecolor='none', alpha=0.5)
    ax.scatter(any_df.x, any_df.y, s=7, edgecolor='none', facecolor='orange', alpha=1.0)

In [ ]:
fig, axs = plt.subplots(5, 4, figsize=(15, 13), gridspec_kw=dict(wspace=0.05))

brain_alpha = 0.5
fs = 14

for subject, (r, c) in zip(lateral_and_medial_subjects, [[0, 0], [0, 2], [1, 0], [1, 2], [2, 0]]):
    
    bad_ch = np.unique(np.concatenate(list(bad_channels[subject].values())))
    edf = all_edf.loc[(all_edf.subject == subject) & (~all_edf.location.isin(exclude_areas))]
    
    ax = axs[r, c]
    view = 'medial'
    cur_edf = edf.loc[(edf['view'] == view) & (~edf.electrode.isin(bad_ch))]
    img = plt.imread(subj_img_path.format(subject, view))
    ax.imshow(img, alpha=brain_alpha)
    ax.annotate(subject, (1, 1.05), xycoords='axes fraction', fontsize=fs)
    
    plot_elecs(ax=ax, df=cur_edf);
    
    ax = axs[r, c+1]
    view = 'lateral'
    cur_edf = edf.loc[(edf['view'] == view) & (~edf.electrode.isin(bad_ch))]
    img = plt.imread(subj_img_path.format(subject, view))
    ax.imshow(img, alpha=brain_alpha)
    
    plot_elecs(ax=ax, df=cur_edf);
    
view = 'lateral'
for subject, ax in zip(lateral_subjects, axs.ravel()[10:]):
    
    bad_ch = np.unique(np.concatenate(list(bad_channels[subject].values())))
    edf = all_edf.loc[(all_edf.subject == subject) & (~all_edf.location.isin(exclude_areas))]
    
    cur_edf = edf.loc[(edf['view'] == view) & (~edf.electrode.isin(bad_ch))]
    img = plt.imread(subj_img_path.format(subject, view))
    ax.imshow(img, alpha=brain_alpha)
    ax.axes.set_title(subject, fontsize=fs)
    
    plot_elecs(ax=ax, df=cur_edf);
    
for ax in axs.ravel():
    ax.axis('off');